In [4]:
filename = 'points_NSX_TMD_32_40_28.json'
In = 28


import numpy as np
import json

def get_relay_curve(In=1, order=4, C=1):
    def relay_curve(I, *ks):
        t = 0
        for order, k in enumerate(ks):
            t += k*np.log10((I/In)-C)**order
        return 10**t
    return relay_curve

with open(filename) as f:
    data = json.load(f)['Points']

x = np.array([row['X'] for row in data])/In
y = np.array([row['Y'] for row in data])
points_n = np.argwhere(x == np.min(x))[0,0]
i_l = x[3:points_n][::-1]
t_l = y[3:points_n][::-1]
i_u = x[points_n+3:]
t_u = y[points_n+3:]

l_filter = t_l <= 3600
u_filter = t_u <= 3600
i_l = i_l[l_filter]
t_l = t_l[l_filter]
i_u = i_u[u_filter]
t_u = t_u[u_filter]

def reduce(i, t):
    i1 = i[0]
    i2 = i[-1]
    i_n = np.geomspace(i1,i2,num=50)
    i_n_nd = np.tile(i_n, (len(i), 1))
    i_nd = np.tile(i, (len(i_n), 1)).T
    idx = (np.abs(i_n_nd - i_nd)).argmin(axis=0)
    return i[idx], t[idx]

i_u, t_u = reduce(i_u, t_u)
i_l, t_l = reduce(i_l, t_l)

In [16]:
import matplotlib.pyplot as plt
%matplotlib auto

plt.plot(i_u, t_u, 'b-', label='upper')
plt.plot(i_l, t_l, 'r-', label='lower')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,100)
plt.ylim(0.1,1e4)
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()

Using matplotlib backend: TkAgg


In [17]:
# Curve fitting upper

In = 1.28
order=7
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_u)
ydata = np.array(t_u)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 25, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[1.7364, -1.63135, 0.15601, 0.44976, -0.22628, -0.34995, 0.06593, 0.10088]

In [18]:
# Curve fitting lower

In=1.03
order=7
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_l)
ydata = np.array(t_l)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 15, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[1.63813, -1.6424, -0.11415, 0.30778, 0.18756, -0.11111, -0.10452, -0.01042]

In [9]:
print(np.min(i_l), np.min(i_u))
print(json.dumps(list(np.round(i_u, 5))), '\n')
print(json.dumps(list(np.round(t_u, 5))), '\n')
print(json.dumps(list(np.round(i_l, 5))), '\n')
print(json.dumps(list(np.round(t_l, 5))), '\n')

1.067857142857143 1.332142857142861
[1.33214, 1.41071, 1.49286, 1.57857, 1.67143, 1.76786, 1.87143, 1.98214, 2.09643, 2.21786, 2.34643, 2.48571, 2.62857, 2.78214, 2.94643, 3.11786, 3.29643, 3.48929, 3.68214, 3.89643, 4.14643, 4.36071, 4.64643, 4.89643, 5.18214, 5.50357, 5.825, 6.14643, 6.50357, 6.89643, 7.28929, 7.71786, 8.18214, 8.64643, 9.14643, 9.68214, 10.25357, 10.825, 11.46786, 12.14643, 12.86071, 13.61071, 14.39643, 15.21786, 16.11071, 17.075, 18.03929, 19.11071, 20.21786, 21.39643] 

[3411.86231, 1413.58531, 806.79439, 520.91387, 362.14169, 253.48009, 189.87348, 145.87827, 112.8092, 90.48946, 73.94823, 60.14719, 50.54742, 42.79824, 36.1743, 30.7661, 26.8061, 23.28866, 20.38649, 17.72421, 15.48269, 13.90297, 12.33405, 11.31694, 10.31055, 9.34034, 8.50903, 7.79063, 7.10053, 6.44853, 5.88773, 5.36046, 4.87011, 4.44811, 4.05585, 3.69392, 3.36194, 3.07547, 2.79748, 2.54547, 2.31747, 2.1115, 1.92559, 1.75788, 1.60076, 1.45501, 1.32949, 1.20931, 1.10248, 1.00453] 

[1.06786, 1.125, 1.